In [23]:
from pandas import read_csv
import tensorflow as tf
import numpy as np


In [37]:
df = read_csv('./features.csv')
df.head()

,ID,Label,Sequence,hydrophobicity_polar_first,hydrophobicity_polar_20%,hydrophobicity_polar_40%,hydrophobicity_polar_60%,hydrophobicity_polar_80%,hydrophobicity_polar_100%,hydrophobicity_neutral_first,...,solvent_accessibility_intermediate_40%,solvent_accessibility_intermediate_60%,solvent_accessibility_intermediate_80%,solvent_accessibility_intermediate_100%,solvent_accessibility_exposed_first,solvent_accessibility_exposed_20%,solvent_accessibility_exposed_40%,solvent_accessibility_exposed_60%,solvent_accessibility_exposed_80%,solvent_accessibility_exposed_100%
0,WP_395757889.1,AMP,MPYDSVYLEKRPPGALRTVWRKFYGDTTAMIGLYGCAGLLLLCVFG...,1.351351,18.918919,30.405405,63.175676,80.405405,100.000000,0.675676,...,37.162162,54.054054,73.648649,93.243243,1.351351,18.918919,30.405405,63.175676,80.405405,100.000000
1,WP_395755974.1,AMP,MKILIVEDDTLIQQGLAQAMARENYAYDCADSAVGASALLQSSHYS...,0.909091,25.454545,44.090909,63.636364,83.181818,99.090909,4.545455,...,37.727273,64.545455,84.545455,100.000000,0.909091,25.454545,44.090909,63.636364,83.181818,99.090909
2,WP_395732316.1,AMP,MDFSRKPTQDPEAWLDFAPEFSQSMARQVRGWIQRWEPDLAESIKW...,0.985222,18.226601,41.871921,60.591133,86.206897,99.507389,1.970443,...,39.901478,63.054187,78.325123,95.073892,0.985222,18.226601,41.871921,60.591133,86.206897,99.507389
3,WP_395723680.1,AMP,MDCSRTPTRDPETWLDLAPEFSQAMARQVRAWIQRWEPDLTESIKW...,0.985222,19.211823,47.783251,65.517241,86.699507,99.507389,1.970443,...,32.512315,49.753695,68.965517,92.118227,0.985222,19.211823,47.783251,65.517241,86.699507,99.507389
4,WP_395695240.1,AMP,MADRDTAGVTFETTVSSFGNNTGIEVPEEVIERLGRGKRPPVLVTV...,1.923077,20.512821,45.512821,71.794872,89.102564,100.000000,1.282051,...,34.615385,53.205128,76.282051,88.461538,1.923077,20.512821,45.512821,71.794872,89.102564,100.000000


In [30]:
df['Label'].replace({'nAMP': 0, 'AMP': 1}, inplace=True)

C:\Users\npchi\AppData\Local\Temp\ipykernel_29064\1046682262.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Label'].replace({'nAMP': 0, 'AMP': 1}, inplace=True)


In [33]:
y = df['Label'].values
y

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [ ]:
# Các tham số
num_amino_acids = 20 
max_length = 200     
amino_acids = "ACDEFGHIKLMNPQRSTVWXY"  
dic_to_index = {val: i + 1 for i, val in enumerate(amino_acids)}  
index_to_dic = {i + 1: val for i, val in enumerate(amino_acids)}

In [36]:
sequences = df['Sequence'].values

# Step 1: Encode the gene sequence into integers (e.g., 'A' -> 1, 'T' -> 2, 'G' -> 3, 'C' -> 4)
# nucleotides = ['A', 'T', 'G', 'C']
# nucleotide_dict = {nucleotide: idx + 1 for idx, nucleotide in enumerate(nucleotides)}

def encode_sequence(sequence):
    return [dic_to_index[nuc] for nuc in sequence]

# Example gene sequences
#sequences = ["ATGC", "GTCA", "GATTACA"]

#labels = [0, 0, 1]

# Encode sequences
encoded_sequences = [encode_sequence(seq) for seq in sequences]
#encoded_sequences
max_length = max([len(seq) for seq in encoded_sequences])
for i in range(len(encoded_sequences)):
    if len(encoded_sequences[i]) < max_length:
        encoded_sequences[i].extend([0] * (max_length - len(encoded_sequences[i])))
X = np.array(encoded_sequences)
#y = np.array(labels)

KeyError: 'X'

In [35]:
X.shape

(3, 7)

In [17]:
# Step 2: Define the model in TensorFlow
# We will use an Embedding layer followed by an LSTM layer to learn sequence-level embeddings

# Parameters
embedding_dim = 8  # Dimensionality of the embedding vector
lstm_units = 16    # Number of LSTM units

# Define the model
model = tf.keras.Sequential([
    # Embedding Layer
    tf.keras.layers.Embedding(input_dim = len(nucleotides) + 1, output_dim = embedding_dim, input_length = max_length),
    
    # LSTM Layer
    tf.keras.layers.LSTM(lstm_units, return_sequences=False),  # return_sequences=False to output a single vector
    
    # Dense Layers for classification (optional, depends on your task)
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Example output layer (for classification)
])

# Step 3: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 4: Train the model (dummy labels for illustration purposes)
# For this example, let's assume the sequences are labeled for a classification task (e.g., 0, 1, 2, 3)
#labels = np.array([0, 1, 2])

# Train the model
model.fit(X, y, epochs=5, batch_size=2)

# Step 5: Get the embeddings for a sequence
sequence = "ATGCAAA"  # Example sequence
encoded_sequence = encode_sequence(sequence)  # Encode the sequence
encoded_sequence = np.expand_dims(encoded_sequence, axis=0)  # Add batch dimension

# Get the embedding output
embedding_output = model.predict(encoded_sequence)

print(f"Embedding for sequence {sequence}: {embedding_output}")

Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2222 - loss: 0.6938   
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7778 - loss: 0.6910 
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6111 - loss: 0.6894 
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6111 - loss: 0.6880 
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6111 - loss: 0.6866 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Embedding for sequence ATGCAAA: [[0.49512506]]


In [21]:
word_embeddings = model.layers[0].get_weights()[0]
word_embeddings

array([[ 0.01905316,  0.00744199, -0.01677316,  0.01889774,  0.05205274,
         0.03196178, -0.00954081,  0.0293442 ],
       [-0.01140689,  0.00232307,  0.02454604,  0.01072338,  0.04322114,
        -0.04160134,  0.02519789, -0.0073183 ],
       [-0.00293613, -0.03492306,  0.01700757,  0.01044878, -0.01957863,
         0.01956827,  0.01042726, -0.05674615],
       [-0.04168224,  0.03075428, -0.04115203,  0.00584471,  0.0204232 ,
         0.00870383, -0.03223594,  0.00554315],
       [ 0.04545868, -0.01904867,  0.01556143,  0.01035725, -0.01934252,
         0.03147958,  0.03312983,  0.03830959]], dtype=float32)

In [ ]:
#elasstic search => 
#search 